<a href="https://colab.research.google.com/github/charlotteesharpp/IMLO-project/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import time

In [22]:
# Define the transform to preprocess the images
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images to 224x224
    transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
    transforms.RandomRotation(15),  # Randomly rotate images in the range (-15 degrees, +15 degrees)
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),  # Randomly change brightness, contrast, saturation, and hue
    transforms.RandomCrop(224, padding=4, padding_mode='reflect'),  # Randomly crop images padded by 4 pixels on each side
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize the images
])

# Define a simpler transform for the validation/test data (no random augmentations)
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [23]:
# Load the dataset (you will need to specify the path to your dataset)
train_dataset = datasets.Flowers102(
    root="data",
    split="train",
    transform=train_transform,
    download=True,
)
test_dataset = datasets.Flowers102(
    root="data",
    split="test",
    transform=test_transform,
    download=True,
)

val_dataset = datasets.Flowers102(
    root="data",
    split="val",
    transform=test_transform,
    download=True,
)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [24]:
class FlowerClassifier(nn.Module):
    def __init__(self):
        super(FlowerClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)


        self.fc1 = nn.Linear(128 * 14 * 14, 2048)
        self.fc2 = nn.Linear(2048, 1024)
        self.fc3 = nn.Linear(1024, 102)

    def forward(self, x):
        x = self.bn1(self.conv1(x))
        x = torch.relu(x)
        x = self.pool(x)

        x = self.bn2(self.conv2(x))
        x = torch.relu(x)
        x = self.pool(x)

        x = self.bn3(self.conv3(x))
        x = torch.relu(x)
        x = self.pool(x)

        # Flatten the tensor for the fully connected layer
        x = x.view(x.size(0), -1)

        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [25]:
# Initialize the model, loss function, and optimizer
model = FlowerClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [26]:
def train_model(model, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f"Epoch [{epoch}/{num_epochs}]: Batch [{batch_idx}/{len(train_loader)}] Loss: {loss.item():.4f}")

def validate_model(model, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for val_images, val_labels in val_loader:
            val_outputs = model(val_images)
            val_loss += criterion(val_outputs, val_labels).item()
    return val_loss

def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for test_images, test_labels in test_loader:
            test_outputs = model(test_images)
            _, predicted = torch.max(test_outputs, 1)
            total += test_labels.size(0)
            correct += (predicted == test_labels).sum().item()
    accuracy = correct / total
    return accuracy

In [27]:
num_epochs = 15
start_time = time.time()

for epoch in range(num_epochs):
        train_model(model, train_loader, optimizer, criterion, epoch)
        if epoch % 5 == 0:
            val_loss = validate_model(model, val_loader, criterion)
            print(f"Validation Loss at Epoch {epoch}: {val_loss:.4f}")

test_accuracy = test_model(model, test_loader)
print(f"Test Accuracy (top-1): {100 * test_accuracy:.2f}%")
print(f"Finished Training. Total elapsed time: {round((time.time() - start_time) / 60, 1)} min")

Epoch [0/15]: Batch [0/32] Loss: 4.6280
Validation Loss at Epoch 0: 148.0071
Epoch [1/15]: Batch [0/32] Loss: 4.6228
Epoch [2/15]: Batch [0/32] Loss: 4.6297
Epoch [3/15]: Batch [0/32] Loss: 4.6257
Epoch [4/15]: Batch [0/32] Loss: 4.6331
Epoch [5/15]: Batch [0/32] Loss: 4.6188
Validation Loss at Epoch 5: 148.0064
Epoch [6/15]: Batch [0/32] Loss: 4.6281
Epoch [7/15]: Batch [0/32] Loss: 4.6263
Epoch [8/15]: Batch [0/32] Loss: 4.6296
Epoch [9/15]: Batch [0/32] Loss: 4.6308
Epoch [10/15]: Batch [0/32] Loss: 4.6249
Validation Loss at Epoch 10: 148.0049
Epoch [11/15]: Batch [0/32] Loss: 4.6301
Epoch [12/15]: Batch [0/32] Loss: 4.6145
Epoch [13/15]: Batch [0/32] Loss: 4.6272
Epoch [14/15]: Batch [0/32] Loss: 4.6250
Test Accuracy (top-1): 0.76%
Finished Training. Total elapsed time: 67.0 min


In [29]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to path")

Saved PyTorch Model State to path
